# Campo Lista Spark

Para usar pyspark en una notebook debemos tener

In [2]:
#pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark
findspark.init()

In [2]:
# Creamos la session de Spark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

### Ejemplo

Les comparto un campo de ejemplo de una tabla con este tipo de estructura, como veran es un string plano

In [3]:
l = [("27200000000","", "2023-05-01"),
     ("27206500000","null", "2023-05-01"),
     ("27206512800","[]", "2023-05-01"),
     ("20330161532",'[{"descripcionActividad":"CULTIVO DE CEREALES N.C.P., EXCEPTO LOS DE USO FORRAJERO","idActividad":11119.0,"nomenclador":883.0,"orden":3.0,"periodo":202105.0},{"descripcionActividad":"VENTA AL POR MENOR DE PRODUCTOS COSMÉTICOS, DE TOCADOR Y DE PERFUMERÍA","idActividad":477320.0,"nomenclador":883.0,"orden":1.0,"periodo":202105.0},{"descripcionActividad":"SERVICIOS DE ASESORAMIENTO, DIRECCIÓN Y GESTIÓN EMPRESARIAL N.C.P.","idActividad":702099.0,"nomenclador":883.0,"orden":2.0,"periodo":202103.0}]',"2023-05-01")]


df_json=spark.createDataFrame(l,["cuit_cuil","actividad_regimen_general","partition_date"])
df_json.show()

+-----------+-------------------------+--------------+
|  cuit_cuil|actividad_regimen_general|partition_date|
+-----------+-------------------------+--------------+
|27200000000|                         |    2023-05-01|
|27206500000|                     null|    2023-05-01|
|27206512800|                       []|    2023-05-01|
|20330161532|     [{"descripcionAct...|    2023-05-01|
+-----------+-------------------------+--------------+



Bueno, ya tenemos la columna en un dataframe (este paso anterior no sera necesario porque ustedes se conectan al lago)

Ahora vamos a pasarle la estructura que deberia tener este texto plano, que es **como queremos que se comporte**

Al verlo intuimos lo siguiente: Una lista de estructuras, la cual se escribe en SPARK de la siguiente forma

In [4]:
schema = ArrayType(StructType(
      [
        StructField('descripcionActividad',  StringType(), True ),
        StructField('idActividad', StringType(), True),
        StructField('nomenclador', StringType(), True),
        StructField('orden', StringType(), True),
        StructField('periodo', StringType(), True)
      ]
    ))

In [5]:
mapped_df = df_json.withColumn("act",from_json("actividad_regimen_general", schema))
mapped_df.show()

+-----------+-------------------------+--------------+--------------------+
|  cuit_cuil|actividad_regimen_general|partition_date|                 act|
+-----------+-------------------------+--------------+--------------------+
|27200000000|                         |    2023-05-01|                null|
|27206500000|                     null|    2023-05-01|                null|
|27206512800|                       []|    2023-05-01|                  []|
|20330161532|     [{"descripcionAct...|    2023-05-01|[{CULTIVO DE CERE...|
+-----------+-------------------------+--------------+--------------------+



In [6]:
mapped_df.printSchema()

root
 |-- cuit_cuil: string (nullable = true)
 |-- actividad_regimen_general: string (nullable = true)
 |-- partition_date: string (nullable = true)
 |-- act: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- descripcionActividad: string (nullable = true)
 |    |    |-- idActividad: string (nullable = true)
 |    |    |-- nomenclador: string (nullable = true)
 |    |    |-- orden: string (nullable = true)
 |    |    |-- periodo: string (nullable = true)



## Desarmar la Lista

Ahora con la funcion `explode` directamente al campo, creara un nuevo registro por cada elemento de la lista

- [explode_outer](https://stackoverflow.com/questions/39739072/spark-sql-how-to-explode-without-losing-null-values) : *Spark 2.2+*

In [7]:
df = mapped_df.withColumn("act_exp", explode_outer("act"))
df.show()

+-----------+-------------------------+--------------+--------------------+--------------------+
|  cuit_cuil|actividad_regimen_general|partition_date|                 act|             act_exp|
+-----------+-------------------------+--------------+--------------------+--------------------+
|27200000000|                         |    2023-05-01|                null|                null|
|27206500000|                     null|    2023-05-01|                null|                null|
|27206512800|                       []|    2023-05-01|                  []|                null|
|20330161532|     [{"descripcionAct...|    2023-05-01|[{CULTIVO DE CERE...|{CULTIVO DE CEREA...|
|20330161532|     [{"descripcionAct...|    2023-05-01|[{CULTIVO DE CERE...|{VENTA AL POR MEN...|
|20330161532|     [{"descripcionAct...|    2023-05-01|[{CULTIVO DE CERE...|{SERVICIOS DE ASE...|
+-----------+-------------------------+--------------+--------------------+--------------------+



Para acceder a los campos dentro de las estructuras

Usamos la notacion punto `("x.y")`   

In [8]:
df.select("cuit_cuil","act_exp.descripcionActividad","act_exp.orden","act_exp.periodo").show()

+-----------+--------------------+-----+--------+
|  cuit_cuil|descripcionActividad|orden| periodo|
+-----------+--------------------+-----+--------+
|27200000000|                null| null|    null|
|27206500000|                null| null|    null|
|27206512800|                null| null|    null|
|20330161532|CULTIVO DE CEREAL...|  3.0|202105.0|
|20330161532|VENTA AL POR MENO...|  1.0|202105.0|
|20330161532|SERVICIOS DE ASES...|  2.0|202103.0|
+-----------+--------------------+-----+--------+



In [9]:
# Si quieren todo pueden usar el siempre u confiable (*)
df.select("cuit_cuil","act_exp.*").show()

+-----------+--------------------+-----------+-----------+-----+--------+
|  cuit_cuil|descripcionActividad|idActividad|nomenclador|orden| periodo|
+-----------+--------------------+-----------+-----------+-----+--------+
|27200000000|                null|       null|       null| null|    null|
|27206500000|                null|       null|       null| null|    null|
|27206512800|                null|       null|       null| null|    null|
|20330161532|CULTIVO DE CEREAL...|    11119.0|      883.0|  3.0|202105.0|
|20330161532|VENTA AL POR MENO...|   477320.0|      883.0|  1.0|202105.0|
|20330161532|SERVICIOS DE ASES...|   702099.0|      883.0|  2.0|202103.0|
+-----------+--------------------+-----------+-----------+-----+--------+

